In [ ]:
import os
import numpy as np
import pyBigWig
import json

# ============================================================
#                  Configuration
# ============================================================
base = "../ML4G_Project_1_Data"
out_config = "../preprocessed_data/global_norm_stats_combined.json"
os.makedirs(os.path.dirname(out_config), exist_ok=True)

marks = ["DNase", "H3K27ac", "H3K4me3", "H3K27me3", "H3K36me3", "H3K4me1", "H3K9me3"]
cells = ["X1", "X2", "X3"]
target_chroms = [f"chr{i}" for i in range(1, 23)]

global_stats = {}

# ============================================================
#                  Main loop: per mark × cell
# ============================================================
for mark in marks:
    for cell in cells:
        # Try both common bigWig extensions
        path_bw = os.path.join(base, f"{mark}-bigwig", f"{cell}.bw")
        if not os.path.exists(path_bw):
            path_bw = os.path.join(base, f"{mark}-bigwig", f"{cell}.bigwig")
        if not os.path.exists(path_bw):
            print(f"⚠️ Missing: {mark} ({cell})")
            continue

        print(f"\n📂 Processing {mark} ({cell}) ...")
        bw = pyBigWig.open(path_bw)
        chroms = bw.chroms()

        # --- Weighted accumulators over all chromosomes (why: compute global mean/std) ---
        total_bp = 0
        weighted_sum_linear = 0.0
        weighted_sq_sum_linear = 0.0
        weighted_sum_log = 0.0
        weighted_sq_sum_log = 0.0

        # ============================================================
        #           Per-chromosome stats in linear and log1p domains
        # ============================================================
        for chrom in target_chroms:
            if chrom not in chroms:
                continue

            vals = np.array(bw.values(chrom, 0, chroms[chrom], numpy=True))
            vals = vals[~np.isnan(vals)]
            n = len(vals)
            if n == 0:
                continue

            # Linear domain statistics
            mean_chr = np.mean(vals)
            std_chr = np.std(vals)

            # Log1p domain (why: stabilize heavy-tailed signals)
            vals_log = np.log1p(np.clip(vals, a_min=0, a_max=None))
            mean_log_chr = np.mean(vals_log)
            std_log_chr = np.std(vals_log)

            # Accumulate weighted sums (why: exact global mean/std via moments)
            total_bp += n
            weighted_sum_linear += n * mean_chr
            weighted_sq_sum_linear += n * (std_chr**2 + mean_chr**2)

            weighted_sum_log += n * mean_log_chr
            weighted_sq_sum_log += n * (std_log_chr**2 + mean_log_chr**2)

        bw.close()

        if total_bp == 0:
            print(f"⚠️ No data found for {mark}-{cell}, skipping.")
            continue

        # ============================================================
        #                  Global (genome-wide) statistics
        # ============================================================
        # Linear domain
        global_mean = weighted_sum_linear / total_bp
        global_var = weighted_sq_sum_linear / total_bp - global_mean**2
        global_std = np.sqrt(global_var)

        # Log1p domain
        global_mean_log1p = weighted_sum_log / total_bp
        global_var_log1p = weighted_sq_sum_log / total_bp - global_mean_log1p**2
        global_std_log1p = np.sqrt(global_var_log1p)

        print(f"✅ {mark}-{cell}:")
        print(f"   • Linear mean/std  = {global_mean:.4f} / {global_std:.4f}")
        print(f"   • Log1p  mean/std  = {global_mean_log1p:.4f} / {global_std_log1p:.4f}")

        # ============================================================
        #                  Persist per mark–cell stats
        # ============================================================
        global_stats[f"{mark}_{cell}"] = {
            "linear": {
                "mean": float(global_mean),
                "std": float(global_std),
                "n_bases": int(total_bp)
            },
            "log1p": {
                "mean": float(global_mean_log1p),
                "std": float(global_std_log1p),
                "n_bases": int(total_bp)
            }
        }

# ============================================================
#                  Save JSON results
# ============================================================
with open(out_config, "w") as f:
    json.dump(global_stats, f, indent=4)

print(f"\n💾 Saved combined normalization stats → {out_config}")



📂 Processing DNase (X1) ...
✅ DNase-X1:
   • Linear mean/std  = 0.0591 / 0.1919
   • Log1p  mean/std  = 0.0530 / 0.0783

📂 Processing DNase (X2) ...
✅ DNase-X2:
   • Linear mean/std  = 0.0406 / 7.5742
   • Log1p  mean/std  = 0.0320 / 0.0926

📂 Processing DNase (X3) ...
✅ DNase-X3:
   • Linear mean/std  = 0.0512 / 4.4270
   • Log1p  mean/std  = 0.0428 / 0.0942

📂 Processing H3K27ac (X1) ...
✅ H3K27ac-X1:
   • Linear mean/std  = 0.5643 / 3.6970
   • Log1p  mean/std  = 0.2519 / 0.3577

📂 Processing H3K27ac (X2) ...
✅ H3K27ac-X2:
   • Linear mean/std  = 0.8620 / 4.5804
   • Log1p  mean/std  = 0.3374 / 0.5282

📂 Processing H3K27ac (X3) ...
✅ H3K27ac-X3:
   • Linear mean/std  = 0.7923 / 5.4479
   • Log1p  mean/std  = 0.3446 / 0.3644

📂 Processing H3K4me3 (X1) ...
✅ H3K4me3-X1:
   • Linear mean/std  = 0.4574 / 3.8905
   • Log1p  mean/std  = 0.1038 / 0.4205

📂 Processing H3K4me3 (X2) ...
✅ H3K4me3-X2:
   • Linear mean/std  = 0.3699 / 3.5539
   • Log1p  mean/std  = 0.0891 / 0.3425

📂 Processin